# About
Phyfleaux is a transformation framework for [Phylanx](https://github.com/stellar-group/phylanx) applications enabling them at "optimizing" execution of *tasks* and providing cost measurement handles  

Phyfleaux is a transformation framework for [Phylanx](https://github.com/stellar-group/phylanx) *task-graph*. Nodes of the graph are *tasks*, i.e., identified Python functions. builds the task-graph from the Phylanx expression-tree (PhySL). *Task* is   order to improve (minimize) the execution time, Phyfleaux transforms s (a.k.a., PhySL) into task-graphs. . application program run as a set of *tasks* on one or more execution environment(s) which we call *context*. A context consists of a task-graph, target architecture(s) (MIMD, SIMD, SIMT, ...), and also one or more runtime systems (interpreters, executors) running the expressions on target architecture(s).

In short, Phyfleaux:

1. generates task-graph 
   - generate the expression-tree (PhySL) 
   - merging expression-trees of Python functions

2. allocates subexpressions to available architecture(s)
   - Minimizing application's execution time by maximizing throughputs of its (Python) functions.

3. generates executable(s), JIT, for context's runtime

4. deploy code along with its data to Agave and Kubernete 
   - through [JetLag](https://github.com/STEllAR-GROUP/JetLag.git), as needed.

5. performance data is collected both at thread-level, through [APEX](https://github.com/khuck/xpress-apex), and task-level for 
<!--
an expression-tree (PhySL),

where each node is a [function definition](https://greentreesnakes.readthedocs.io/en/latest/nodes.html#FunctionDef) and edges represent data dependency between two tasks.
- There is a cost function associated to each task 
- Measurements returned by the cost functions are used to maximize applications computational throughput

Phyfleaux:

1. *statically* generates program's expression-tree (PhySL) __(,  a.k.a., *task-graph/tree*)?__- here by statically we mean any transformation applied after function definition but before its first invokation. or enforced code-generation ("compile-time transformations"?).

2. based on context's tasks and resources, Phyfleaux applies series of transformations to improve program's throughput. Transformation may be triggered by the user, implicitly or explicitly (), . selected by framework after analyzing data (type, layout, ...) and tasks (parallel, iterative, ...) of the application,  

3. For each runtime, Phyfleaux generates executable(-code?) of the invoked tasks based on target contexts.

There are two classes of transformations:

1. Data transformation
2. Computation transformation

Two classes of transformation are available. First, task transformations

### Create Tasks
`@task` is used to instantiate a task from a Python function


## Context
- architecture: parallel
CUDA
OPENMP
SIMD

GPU
CPU 


## Task
- wraps Python functions in *tasks*
- builds the expression-tree of these tasks (from Python AST)
- if neccessary, applies transformations based on (a) invokation context and/or (b) dynamic performance measurements
- generates the executable JIT: either once the task (Python function) is called, or the user triggered the cod
To Do:
- deployment
   * kubernetes
   * Agave
- performance
   * APEX/Phylanx
   * adaptivity
- visualization
   * performance data
   * task graph
-->

In short, Phyfleaux aims at maximizing throughput of Python application by exploiting application's, explicit and implicit, concurrency and parallelism. in the application running as many massively-parallel tasks as possible.

# Getting Started
Let's first make sure `phyfleaux` is on $PYTHONPATH:

In [ ]:
# If running from a ste||ar-container (https://github.com/rtohid/docker-build/blob/master/phyfleaux/Dockerfile.pytiramisu)
STELLAR_CONTAINER = True
if STELLAR_CONTAINER:
    sys.path.append('/home/stellar/git/phyfleaux/')
import phyfleaux

## Task

<!-- Code blocks (essentially Python functions) with their associated Python AST (PAST) and corresponding expression-tree (PhySL). An easy way to create a :class:`Task` object is to use `@task`. Here is a task created from :func:`nothing` which does nothing. -->

In [ ]:
from phyfleaux.api.directives import task

@task
def nothing():
    pass

type(nothing)

Tasks are Python functions associated with a memory-independent ID `id`, an AST `tree`, input data (to initiate the local symbol table) `input`, and a callback function (quantifying execution cost) `cost` as well as its derivative `derivative` (value, or yet another callback function).

In [ ]:
def nothing():
    pass

# attributes of the pure Python function
python_attributes = nothing.__dir__()

@task
def nothing():
    pass

# Task attributes
fleaux_attributes = nothing.__dir__()

print(set(fleaux_attributes) - set(python_attributes))

print(nothing.__weakref__)

t = task(nothing)

In [ ]:
@task
def foo(name='World'):
    print(f"Hello {name}!")

In [ ]:
print(type(foo))